In [ ]:
%pip install pandas
%pip install langchain_community

In [ ]:
import pandas as pd
from langchain_community.llms import Ollama

In [ ]:
llm = Ollama(model="risk_analizer_3.1")
llm.invoke("Pérdida, hurto o deterioro de un activo por un acceso no autorizado al área del Archivo")

In [ ]:
df = pd.read_csv('data.csv')
df

In [ ]:
def categorizar_riesgo(risk, model="risk_analizer_llama3",contador=0, palabrasMaximas=2, largoMaximo=30):
    llm = Ollama(model=model)
    response = llm.invoke(risk).split("\n")[0]
    
    if contador > 3:
        return "No se pudo generar una respuesta" 

    if len(response.split(" ")) > palabrasMaximas:
        print(f"Respuesta mayor a {palabrasMaximas} palabras: Contador: {contador} - {risk} - {response}")
        return categorizar_riesgo(risk, model=model, contador=contador+1, palabrasMaximas=palabrasMaximas, largoMaximo=largoMaximo)
    
    if len(response) > largoMaximo:
        print(f"Respuesta muy larga: Contador: {contador} - {risk} - {response}")
        return categorizar_riesgo(risk, model=model, contador=contador+1, palabrasMaximas=palabrasMaximas, largoMaximo=largoMaximo)
    
    valoresMalos = ["alto", "bajo", "falso"]
    if any(valor in response.lower() for valor in valoresMalos):
        print(f"Respuesta tiene valores que no coinciden: Contador: {contador} - {risk} - {response}")
        return categorizar_riesgo(risk, model=model, contador=contador+1, palabrasMaximas=palabrasMaximas, largoMaximo=largoMaximo)

    # Limpieza de caracteres no deseados
    response = response.replace("*", "").replace(".", "").replace(":", "").replace('"', "").title()
    
    print(f"Respuesta normal: {response}")
    return response
    


In [ ]:
# Leer archivo
df = pd.read_csv('data_clasificada_v2.csv')
# recorrer la columna de riesgos y categorizar el riesgo
for i in range(len(df)):
    
    # Categorizar el riesgo
    resultado = categorizar_riesgo(df["Riesgo"][i], model="risk_analizer_description_llama3", palabrasMaximas=15, largoMaximo=50)
    
    #Guardar la clasificacion en una nueva columna
    df.loc[i, "Descripción"] = resultado
    
# Imprimir el dataframe con la nueva columna
df

# Guardar los resultados en un nuevo archivo
df.to_csv('data_clasificada_v3.csv', index=False)

In [ ]:
# Generar un archivo solo con las clasificaciones
df_clasificacion = df["Clasificación"]
df_clasificacion.to_csv('clasificacion.csv', index=False)

# Leer el archivo con las clasificaciones y agregarlo a un string
clasificacion = ""
with open('clasificacion.csv', 'r') as file:
    clasificacion = file.read()
    
clasificacion

# Enviarlo al modelo para que lo clasifique
llm = Ollama(model="llama3")
response = llm.invoke("Analizar todas las categorias de riesgo y crear 10 categorias generales en las que se puedan clasificar cada una de esas subclasificaciones " + clasificacion)
response = response.split("\n")
response


In [ ]:
# Crear una categoria macro para cada categoria
df = pd.read_csv('data_clasificada.csv')
# recorrer la columna de riesgos y categorizar el riesgo
for i in range(len(df)):
    
    # Categorización
    resultado = categorizar_riesgo(df["Clasificación"][i], model="risk_analizer_macro_llama3")
    
    #Guardar la clasificacion en una nueva columna
    df.loc[i, "Clasificación_Macro"] = resultado
    
# Imprimir el dataframe con la nueva columna
df

# Guardar los resultados en un nuevo archivo
df.to_csv('data_final.csv', index=False)